# ***<u>librariess***

In [109]:
import numpy as np
from itertools import product
from collections import defaultdict

# ***<u>Generate <u>Sudoku***

In [110]:
def generate_sudoku():
    """
    Function to generate a Sudoku grid


    Fill the diagonal boxes of the Sudoku grid
    Solve the Sudoku grid
    Remove elements from the grid to create a puzzle
    """

    
    grid = np.array([[0 for i in range(9)] for j in range(9)])
    fill_diagonal(grid)
    row, column = find_unassigned_location(grid)
    DFS_solve_sudoku(grid, row, column)
            
    emptyCellsNum = np.random.randint(0, 80)
    print("Number of empty cells : {em}\n".format(em=emptyCellsNum))
    remove_elements(grid, emptyCellsNum)
    return grid


def DFS_solve_sudoku(grid,row, col):
    if row == -1 and col == -1:
        return True
    for num in range(1, 10):
        if is_safe(grid, row, col, num):
            grid[row, col] = num
            nextRow, nextColumn = find_unassigned_location(grid)
            if DFS_solve_sudoku(grid, nextRow, nextColumn):
                return True
    
    grid[row, col] = 0
    return False


def fill_diagonal(grid):
    """
    Function to fill the diagonal boxes of the Sudoku grid
    """
    for i in range(0, 9, 3):
        fill_box(grid, i, i)


def fill_box(grid, row, col):
    """
    Function to fill a box with random values
    """
    
    for i in range(3):
        for j in range(3):
            randNum = np.random.randint(1, 10)
            while used_in_box(grid,row,col,randNum):
                randNum = (randNum) % 9 + 1
            grid[row+i, col+j] = randNum
    
    return True


def is_safe(grid, row, col, num):
    """
    Function to check if it is safe to place a number in a particular position
    """

    return (
        not used_in_row(grid, row, num)
        and not used_in_column(grid, col, num)
        and not used_in_box(grid, row - row % 3, col - col % 3, num)
    )


#
def used_in_row(grid, row, num):
    """
    Function to check if a number is used in a row
    """
    return num in grid[row, :]


def used_in_column(grid, col, num):
    """
    Function to check if a number is used in a column
    """
    return num in grid[:, col]


def used_in_box(grid, box_start_row, box_start_col, num):
    """
    Function to check if a number is used in a 3x3 box
    """

    for i in range(3):
        for j in range(3):
            if grid[box_start_row + i,box_start_col+ j]==num:
                return True
    return False

def find_unassigned_location(grid):
    """
    Function to find an unassigned location in the grid
    """

    for i in range(9):
        for j in range(9):
            if grid[i, j] == 0:
                return (i, j)
    return (-1, -1)


def remove_elements(grid, num_elements):
    
    """
    Function to remove elements from the grid
    """

    for i in np.random.choice(81, num_elements, replace=False):
        grid[i // 9, i % 9] = 0

# ***<u>BackTracking***

In [111]:
backTrackingSteps=0

def solve_sudoku(grid):

    """
        Function to solve the Sudoku grid using backtracking
    """
    
    row,col=find_unassigned_location(grid)
    global backTrackingSteps
    if(row == -1 and col == -1):
        print("This grid is solved in {step} steps.\n".format(step=backTrackingSteps))
        return True
    
    backTrackingSteps+=1
    for num in range(1,10):
        if is_safe(grid,row,col,num):
            grid[row,col]=num
            if solve_sudoku(grid):
                return True
            grid[row,col]=0
    return False

        
    

def display_grid(grid):

    """
        Function to display the Sudoku grid
    """

    for i in range(9):
        for j in range(9):
            print(grid[i][j], end=" ")
        print()


# ***<u>CSP***

In [112]:
def solve_sudoku_csp(grid):
    """
    Function to solve the Sudoku grid using Constraint Satisfaction Problem (CSP)
    """
    
    def create_domains(grid):
        """
        Function to create domains for each cell in the grid
        """
        domains = {}
        for row in range(9):
            for col in range(9):
                if grid[row, col] == 0:
                    domains[(row, col)] = []
                    for num in range(1, 10):
                        if is_safe(grid, row, col, num):
                            domains[(row, col)].append(num)
                else:
                    domains[(row, col)] = grid[row, col]
        return domains

    def is_valid_assignment(i, j, val, assignment):
        """
        Function to check if assigning a value to a cell is valid

        Check if the value is already used in the same row
        Check if the value is already used in the same column
        Check if the value is already used in the same 3x3 box
        """

        for row in range(i - i % 3, i - i % 3 + 3):
            for col in range(j - j % 3, j - j % 3 + 3):
                if type(assignment[(row, col)]) == list:
                    continue
                if assignment[(row, col)] == val:
                    return False

        for row in range(9):
            if type(assignment[(row, j)]) == list:
                continue
            if assignment[(row, j)] == val:
                return False

        for col in range(9):
            if type(assignment[(i, col)]) == list:
                continue
            if assignment[(i, col)] == val:
                return False

        return True

    def find_unassigned_location(assignment):
        """
        Function to find an unassigned location in the grid
        """

        for i in range(9):
            for j in range(9):
                if type(assignment[(i, j)]) == list:
                    return (i, j)
        return (-1, -1)

    def shrinkDomain(assignment,row,column,val):
        for i in range(9):
            if type(assignment[(i,column)])==list:
                if val in assignment[(i,column)]:
                    assignment[(i,column)].remove(val)
                    if not assignment[(i,column)]:
                        return False
        
        for j in range(9):
            if type(assignment[(row,j)])==list:
                if val in assignment[(row,j)]:
                    assignment[(row,j)].remove(val)
                    if not assignment[(row,j)]:
                        return False
        
        for i in range(3):
            for j in range(3):
                if type(assignment[(row-row%3+i,column-column%3+j)])==list:
                    if val in assignment[(row-row%3+i,column-column%3+j)]:
                        assignment[(row-row%3+i,column-column%3+j)].remove(val)
                        if not assignment[(row-row%3+i,column-column%3+j)]:
                            return False
        return True
                    
    def extendDomain(assignment,row,column,val):
        for i in range(9):
            if type(assignment[(i,column)])==list and is_valid_assignment(i,column,val,assignment) and  val not in assignment[(i,column)]:
                assignment[(i,column)].append(val)
                assignment[(i,column)].sort()
        for j in range(9):
            if type(assignment[(row,j)])==list and is_valid_assignment(row,j,val,assignment) and val not in assignment[(row,j)]:
                assignment[(row,j)].append(val)
                assignment[(row,j)].sort()
        for i in range(3):
            for j in range(3):
                if ( type(assignment[(row-row%3+i,column-column%3+j)])==list and is_valid_assignment(row-row%3+i,column-column%3+j,val,assignment)
                    and val not in assignment[(row-row%3+i,column-column%3+j)]):
                    assignment[(row-row%3+i,column-column%3+j)].append(val)
                    assignment[(row-row%3+i,column-column%3+j)].sort()
    # Recursive function to solve the Sudoku grid using CSP
    def solve_csp(assignment):
        global cspSteps
        i, j = find_unassigned_location(assignment)
        if i == -1 and j == -1:
            print("This is grid is solved in {step} steps.\n".format(step=cspSteps))
            return True
        cspSteps+=1
        # Try assigning each possible value to the unassigned location
        assignment_copy = assignment[(i, j)].copy()
        for val in assignment[(i, j)].copy():
            if is_valid_assignment(i, j, val, assignment):
                assignment[(i, j)] = val
                if shrinkDomain(assignment,i,j,val):
                    if solve_csp(assignment):
                        return True
                assignment[(i, j)]=0
                extendDomain(assignment,i,j,val)
            assignment[(i, j)] = assignment_copy
        assignment[(i, j)] = assignment_copy
        return False

    global cspSteps
    
    cspSteps=0
    # Create initial domains for each cell in the grid
    domains = create_domains(grid)
    assignment = {(i, j): val for (i, j), val in domains.items()}

    # Solve the Sudoku grid using CSP
    if solve_csp(assignment):
        solved_grid = [[assignment[(i, j)] for j in range(9)] for i in range(9)]
        return solved_grid
    else:
        return None

# ***<u>Show <u>Result***

In [113]:
# Function to initialize the Sudoku grid
def initializing_grid():
    print("\nInitial Sudoku\n")
    sudoku_grid = generate_sudoku()
    for i in sudoku_grid:
        for j in i:
            print(j, end=' ')
        print("")
    print(end='\n\n\n\n\n')
    return sudoku_grid


# Function to solve the Sudoku grid using backtracking
def backtracking_answer(sudoku_grid):
    print("Back Tracking Answer:\n\n")
    if solve_sudoku(sudoku_grid):
        print("Sudoku solved successfully:")
        display_grid(sudoku_grid)
    else:
        print("No solution exists for the given Sudoku.")
    print(end='\n\n\n\n\n')


# Function to solve the Sudoku grid using CSP
def csp_answer(sudoku_grid):
    print("CSP Answer:\n")
    solved_grid = solve_sudoku_csp(sudoku_grid)
    
    if solved_grid is not None:
        print("Sudoku solved successfully:")
        for row in solved_grid:
            for r1 in row:
                print(r1,end=' ')
            print()
    else:
        print("No solution exists for the given Sudoku.")
    print(end='\n\n\n\n\n')

In [114]:
# Generate and display the initial Sudoku grid
generate_sudoko = initializing_grid()


# Solve the Sudoku grid using backtracking
backtracking_answer(generate_sudoko.copy())

# Solve the Sudoku grid using CSP
csp_answer(generate_sudoko)


Initial Sudoku

Number of empty cells : 52

0 0 3 0 0 0 0 0 0 
0 4 0 0 0 0 0 3 9 
0 0 0 1 0 0 0 0 5 
3 0 4 0 0 6 0 0 0 
0 0 9 3 8 0 0 0 0 
0 0 8 0 0 0 3 0 2 
7 0 0 8 4 0 5 9 0 
4 0 1 7 0 0 0 2 0 
5 8 0 9 0 0 0 7 6 





Back Tracking Answer:


This grid is solved in 539 steps.

Sudoku solved successfully:
1 2 3 4 5 9 6 8 7 
8 4 5 6 2 7 1 3 9 
9 6 7 1 3 8 2 4 5 
3 1 4 2 7 6 9 5 8 
2 5 9 3 8 1 7 6 4 
6 7 8 5 9 4 3 1 2 
7 3 6 8 4 2 5 9 1 
4 9 1 7 6 5 8 2 3 
5 8 2 9 1 3 4 7 6 





CSP Answer:

This is grid is solved in 64 steps.

Sudoku solved successfully:
1 2 3 4 5 9 6 8 7 
8 4 5 6 2 7 1 3 9 
9 6 7 1 3 8 2 4 5 
3 1 4 2 7 6 9 5 8 
2 5 9 3 8 1 7 6 4 
6 7 8 5 9 4 3 1 2 
7 3 6 8 4 2 5 9 1 
4 9 1 7 6 5 8 2 3 
5 8 2 9 1 3 4 7 6 





